In [14]:
using LinearAlgebra
using DataFrames
using CSV
using Plots
include("structs.jl")
include("trees.jl")
include("sourceQt.jl")
include("sourceCl.jl");

In [ ]:
function do_all_paulimeasurement(ms, chann, qt)
    Vectors = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
            [1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1],
            [1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]]
    States = [state_vector(i./sqrt(sum(i)), "Z") for i in Vectors];
    Measures = [pauli_proj_meas[i][j] for (i,j) in zip(ms, qt)]
    ch = channel(kron(gates[chann], Matrix(I, 2, 2)), "Z")
    n = 100000
    file = "/home/users/ciolli/Desktop/Code/Rotations/ch-$(chann)_measurement-$(ms[1])$(qt[1])_$(ms[2])$(qt[2])_$(ms[3])$(qt[3])_$(ms[4])$(qt[4]).csv"
    df = DataFrame()
    df.θ = Float64[]
    df.Initial_State = [];

    for state in States
        Qt = QT_Dynamical_model([[-1,1]],[[-1,1]],state, ch, Measures)
        Source, Target = simulation(Qt, n)
        T = Time_Series(n, [Qt.x;Qt.x], [Qt.y;Qt.y], Target, Source, Qt)
        push!(df.TE, TE(2, 1, 1 ,T, Freq)[1].value)
        push!(df.Initial_State, state.vec)
    end
    
    CSV.write(file, df);
end

In [22]:
Vectors = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
[1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1],
[1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]]
States = [state_vector(i./sqrt(sum(i)), "Z") for i in Vectors];
Circuit = [pauli_proj_meas["Mx"]["A"], pauli_proj_meas["Mz"]["B"], channel(kron(gates["PauliX"], Matrix(I,2,2)), "Z"),pauli_proj_meas["Mz"]["A"], pauli_proj_meas["Mz"]["B"]];

In [23]:
sm = create_tree(Circuit,density_matrix(States[1]) ,dummyBinary_tree())

Binary_tree(1, nothing, density_prob(1.0, ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im], 0.0), Union{Nothing, Binary_tree}[Binary_tree(2, Binary_tree(#= circular reference @-3 =#), density_prob(0.4999999999999998, ComplexF64[0.5 + 0.0im 0.0 + 0.0im -0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.5 + 0.0im 0.0 + 0.0im 0.5 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im], -1.0), Union{Nothing, Binary_tree}[Binary_tree(4, Binary_tree(#= circular reference @-3 =#), density_prob(0.0, ComplexF64[0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im], 0.0), Union{Nothing, Binary_tree}[Binary_tree(8, Binary_tree(#= circular reference @-3 =#), dens

In [24]:
trimming!(sm)

In [31]:
measures(sm)

4-element Vector{Float64}:
 -1.0
  1.0
 -1.0
  1.0

In [ ]:
function doall_rotations(chanl,rotations, n, word)
    Vectors = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
            [1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1],
            [1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]]
    States = [state_vector(i./sqrt(sum(i)), "Z") for i in Vectors];
    df = DataFrame()
    chann = gates[chanl]
    file = "/home/users/ciolli/Desktop/Code/Rotations/ch-$(chanl)_$(word).csv"
    if size(chann)[1] == 4
        ch = channel(chann, "Z")
    elseif size(chann)[1] == 2
        ch = channel(kron(chann, Matrix(I, 2, 2)), "Z")
    end
    mB = pauli_proj_meas["Mz"]["B"]
    i = 0
    for state in States
        i += 1
        J = []
        for mA in rotations
            Circuit = [mA, mB, ch, mA, mB];
            Qt = QT_Dynamical_model([[mA.value[1],mA.value[2]]],[[mB.value[1],mB.value[2]]],state, Circuit)
            Source, Target = simulation(Qt, n)
            T = Time_Series(n, [Qt.x;Qt.x], [Qt.y;Qt.y], Target, Source, Qt)
            push!(J, TE(2, 1, 1 ,T, Freq)[1].value)
        end
        df[:, string(Vectors[i])] = J
    end

    CSV.write(file, df);
    return
end

In [ ]:
nmeas = 30
rotations = [measure(operator(rot(σy,σz, -(i/nmeas)*π/2), "Z"), 1, 2) for i in 0:nmeas]
channe = "Swap"
n = 100000;
i = 0
for (key, value) in rotaz
    i += 1
    doall_rotations(channe, value, n, key);
    println(i)
end